In [1]:
#Adopting TREX (RatzEtAl Nat Neuro 2022) data for ZhangMelzerEtAl 2023
#Last updated 20231003 by Madeline E Melzer

In [1]:
#import csv
#import gzip
import os
import pandas as pd
#import scipy.io
#import pyreadr

In [8]:
inputDirectory = "/Users/mem3579/Documents/GitHub/TREX/trex_singletCode/"
outputDirectory = "/Users/mem3579/Library/CloudStorage/OneDrive-NorthwesternUniversity/Arispe and Goyal Labs/ZhangMelzerEtAl/data/CJ/"
outputDirectory2 = "/Users/mem3579/Library/CloudStorage/OneDrive-NorthwesternUniversity/Arispe and Goyal Labs/ZhangMelzerEtAl/data/CJ/formatted/"
#outputDirectory2 = "/Users/mem3579/Library/CloudStorage/OneDrive-NorthwesternUniversity/Arispe and Goyal Labs/ZhangMelzerEtAl/data/TREX/cloneID/formatted2/"

#format_umi_count_matrices(outputDirectory, outputDirectory2)

#append_csv_files(outputDirectory, os.path.join(outputDirectory, "all_brains_umi_count_matrix.csv"))

expand_rows_based_on_count(outputDirectory, outputDirectory2)


In [25]:
dfs = []

for filename in os.listdir(outputDirectory2):
    if filename.startswith("brain") and filename.endswith(".csv"):
        file_path = os.path.join(outputDirectory2, filename)

        # Extract sample number from filename
        sample_number = filename.split("_")[0].replace("brain", "")

        # Read the .csv file into a DataFrame
        df = pd.read_csv(file_path)

        # Add the sample column
        df['sample'] = sample_number

        # Append to the list of DataFrames
        dfs.append(df)

# Concatenate all DataFrames together
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to a new .csv file
combined_output_path_script = os.path.join(outputDirectory2, "all_brains_umi_count_matrix.csv")
combined_df.to_csv(combined_output_path_script, index=False)

# Display the first few rows of the combined data for verification
combined_df.head()

,cellID,barcode,sample
0,10x52_AAACCCAGTAGCCAGA,GTGTGCAGCTTTGAAGGGTGATGTGGGGGG,3
1,10x52_AAACCCAGTAGCCAGA,GTGTGCAGCTTTGAAGGGTGATGTGGGGGG,3
2,10x52_AAACCCAGTAGCCAGA,TTGGCGGAGCGGGGGGAGGGAGGGCGATAC,3
3,10x52_AAACGCTAGCATGATA,TATTGTGCCGCCCGAGAGTAGCGTGGGGGC,3
4,10x52_AAACGCTAGCATGATA,TATTGTGCCGCCCGAGAGTAGCGTGGGGGC,3


In [2]:
def format_umi_count_matrices(input_folder, output_folder):
    """
    Transform UMI count matrices from all CSV files in a given folder and save the transformed data to new CSV files in the output folder.

    Parameters:
    - input_folder: Path to the input folder containing CSV files.
    - output_folder: Path where the transformed CSV files will be saved.

    Returns:
    - None
    """

    # Ensure the output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            # Construct full paths for input and output files
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename.replace(".csv", "_formatted.csv"))

            # Load the CSV file into a DataFrame
            df = pd.read_csv(input_path, index_col=0)

            # Melt the DataFrame to reshape it
            melted_df = df.reset_index().melt(id_vars='index', var_name='cellID', value_name='count')

            # Rename columns and reorder them
            melted_df = melted_df.rename(columns={'index': 'barcode'})
            melted_df = melted_df[['cellID', 'barcode', 'count']]

            # Remove rows where count is 0
            filtered_df = melted_df[melted_df['count'] != 0].reset_index(drop=True)

            # Save the filtered data to the specified output path
            filtered_df.to_csv(output_path, index=False)

In [12]:
def append_csv_files(input_folder, output_path):
    """
    Append all .csv files in a given folder into one .csv file.

    Parameters:
    - input_folder: Path to the input folder containing the .csv files.
    - output_path: Path where the combined .csv file will be saved.

    Returns:
    - None
    """

    # List to hold individual DataFrames
    dfs = []

    # Iterate over each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_folder, filename)

            # Read the .csv file into a DataFrame and append to the list
            df = pd.read_csv(file_path)
            dfs.append(df)

    # Concatenate all DataFrames together
    combined_df = pd.concat(dfs, ignore_index=True)

    # Save the combined DataFrame to the specified output path
    combined_df.to_csv(output_path, index=False)

In [7]:
def expand_rows_based_on_count(input_folder, output_folder):
    """
    For each .csv file in the input folder ending with "_formatted.csv", duplicate rows based on the "count" value
    and then drop the "count" column. The processed data is saved to the specified output folder with the same filename.

    Parameters:
    - input_folder: Path to the folder containing the .csv files.
    - output_folder: Path to the folder where the processed files will be saved.

    Returns:
    - None
    """

    # Ensure the output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith("_formatted.csv"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # Read the .csv file into a DataFrame
            df = pd.read_csv(input_path)

            # Duplicate rows based on the "count" column
            df = df.loc[df.index.repeat(df['count'])]

            # Drop the "count" column
            df = df.drop(columns=['count']).reset_index(drop=True)

            # Save the processed DataFrame to the specified output path
            df.to_csv(output_path, index=False)
